In [1]:
import numpy as np
from numba import jit
import torch

import timeit

In [2]:
def dist_vec(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    dist = np.linalg.norm(
        x[:, None, :] - x[None, :, :],
        axis=-1)
    vecs = x[None, :, :] - x[:, None, :]       
    return dist, vecs

In [3]:
@jit
def dist_jit(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    dist = np.zeros((x.shape[0], x.shape[0]))
    vecs = np.zeros((x.shape[0], x.shape[0], x.shape[-1]))
    for i in range(x.shape[0]):
        for j in range(i):
            dist[i][j] = np.linalg.norm(x[i] - x[j])
            vecs[i][j] = x[i] - x[j]
            dist[j][i] = dist[i][j]
            vecs[j][i] = vecs[i][j]
    return dist, vecs

In [4]:
def dist_torch(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x)
    #dist = torch.zeros((x.shape[0], x.shape[0]))
    #vecs = torch.zeros((x.shape[0], x.shape[0], x.shape[-1]))
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1), vecs

In [5]:
def dist_cuda(x): 
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x).cuda()
    #dist = torch.zeros((x.shape[0], x.shape[0]))
    #vecs = torch.zeros((x.shape[0], x.shape[0], x.shape[-1]))
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1).cpu(), vecs.cpu()

In [6]:
x = np.array([[0., 0.],
                [0., 1.],
                [1., 1.],
                [1., 0.]])
q = np.array([1., -1., 1., -1.])

In [7]:
assert (dist_vec(x)[0] == dist_jit(x)[0]).any
assert (dist_vec(x)[1] == dist_jit(x)[1]).any
assert (dist_vec(x)[0] == dist_torch(x)[0].numpy()).any
assert (dist_vec(x)[1] == dist_torch(x)[1].numpy()).any
#assert (dist_vec(x)[0] == dist_cuda(x)[0].numpy()).any
#assert (dist_vec(x)[1] == dist_cuda(x)[1].numpy()).any

In [8]:
import random

def Random_particles(N):
    """Creates a list of N particles with random positions and charges
    
    Arguments:
        N (int): number of particles
        
    Output:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
    """
    return np.random.rand(N,3), np.array([[-1,1][random.randrange(2)] for i in range(N)])


In [9]:
N = 500
x, q = Random_particles(N)

In [10]:
%timeit dist_torch(x)
#%timeit dist_cuda(x)
%timeit dist_vec(x)
%timeit dist_jit(x)

5.72 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
31.2 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.27 s ± 36.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
